### Project 1

In this project our goal is to validate one dictionary structure against a template dictionary.

A typical example of this might be working with JSON data inputs in an API. You are trying to validate this received JSON against some kind of template to make sure the received JSON conforms to that template (i.e. all the keys and structure are identical - value types being important, but not the value itself - so just the structure, and the data type of the values).

To keep things simple we'll assume that values can be either single values (like an integer, string, etc), or a dictionary, itself only containing single values or other dictionaries, recursively. In other words, we're not going to deal with lists as possible values. Also, to keep things simple, we'll assume that all keys are **required**, and that no extra keys are permitted.

In practice we would not have these simplifying assumptions, and although we could definitely write this ourselves, there are many 3rd party libraries that already exist to do this (such as `jsonschema`, `marshmallow`, and many more, some of which I'll cover lightly in some later videos.)

For example you might have this template:

In [1]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

So, a JSON document such as this would match the template:

In [2]:
john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}

But this one would **not** match the template (missing key):

In [3]:
eric = {
    'user_id': 101,
    'name': {
        'first': 'Eric',
        'last': 'Idle'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 3,
            'day': 29
        },
        'birthplace': {
            'country': 'United Kingdom'
        }
    }
}

And neither would this one (wrong data type):

In [4]:
michael = {
    'user_id': 102,
    'name': {
        'first': 'Michael',
        'last': 'Palin'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 'May',
            'day': 5
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Sheffield'
        }
    }
}

Write a function such this:

In [5]:
def validate(data, template):
    # implement
    # and return True/False
    # in the case of False, return a string describing 
    # the first error encountered
    # in the case of True, string can be empty
    def recurse_validate(data, template, path=''):
        if path:
            path += '.'
        # check if data has extra key(s)
        if (extra_keys := data.keys() - template.keys()):
            raise Exception(f'extra keys: '+ ', '.join(f'{path}{k}' for k in extra_keys))
        # check if data has missing key(s)
        if (missing_keys := template.keys() - data.keys()):
            raise Exception(f'missing keys: '+ ', '.join(f'{path}{k}' for k in missing_keys))
        for k, datatype in template.items():
            key_name = f'{path}{k}'
            try:
                data_to_check = data[k]
            except KeyError:
                raise Exception(f'missing key: {key_name}') from None
            if isinstance(datatype, dict):
                if isinstance(data_to_check, dict):
                    recurse_validate(data_to_check, datatype, key_name)
                else:
                    raise Exception(f'incorrect type: {key_name} -> expected dict, found {type(data_to_check).__name__}')
            elif not isinstance(data[k], datatype):
                raise Exception(f'incorrect type: {key_name} -> expected {datatype.__name__}, found {type(data_to_check).__name__}')
    recurse_validate(data, template)
    return True

That should return this:
* `validate(john, template) --> True, ''`
* `validate(eric, template) --> False, 'mismatched keys: bio.birthplace.city'`
* `validate(michael, template) --> False, 'bad type: bio.dob.month'`

In [6]:
validate(john, template)

True

In [7]:
validate(eric, template)

Exception: missing keys: bio.birthplace.city

In [8]:
validate(michael, template)

Exception: incorrect type: bio.dob.month -> expected int, found str

Better yet, use exceptions instead of return codes and strings!